In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3' # tensorflow 실행시 출력되는 정보 메시지 지우기
import warnings
warnings.filterwarnings('ignore') # keras 에서 발생하는 warning 지우기
#위에 둘은 굳이 안 없애도 되지만 출력이 너무 지저분해서 작성함. 없애도 됨.

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import RMSprop

# early stopping 을 위함
from tensorflow.keras.callbacks import EarlyStopping
# train data 와 test data 분할을 위함
from sklearn.model_selection import train_test_split

# 각각 sampling 에 사용되는 함수들을 import
# 모두 imblearn 라이브러리에 포함됨
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import CondensedNearestNeighbour 

In [3]:
# 데이터 셋 읽어오기
df= pd.read_csv('machine_failure_cleaned.csv')

In [4]:
# Unused
df = df.drop(columns=['TWF', 'HDF', 'PWF', 'OSF']) # 필요없는 feature 제거 
data = df.to_numpy() #numpy 로 바꿔서 data 에 저장 

# 학습에 사용된 데이터 일부 출력해서 확인
print(data[:5])

[[1551.    42.8    0.     0. ]
 [1408.    46.3    3.     0. ]
 [1498.    49.4    5.     0. ]
 [1433.    39.5    7.     0. ]
 [1408.    40.     9.     0. ]]


In [5]:
x_data = df.drop(columns=['Machine failure']).values # Machine failure 부분은 y값에 사용할거니까 feature에서는 없애고 
y_data = df['Machine failure'].values # y 값에 저장 

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42, stratify=y_data
    ) # 데이터 split / test_size는 전체의 20%로 했습니다 

In [6]:
# 원래 0 이랑 1이 몇개였는지 확인
print("sampling 전 클래스 분포:") 
unique, counts = np.unique(y_train, return_counts=True) 
for label, count in zip(unique, counts): #반복문에 넣고 출력 
    print(f" {label}: {count}개")
    # 클래스 1이 극단적으로 적은 불균형 데이터임을 확인

sampling 전 클래스 분포:
 0: 7623개
 1: 229개


In [7]:
# 다수클래스가 7000개라면 소수클래스를 1500개까지 늘리겠다는 부분
# 소수클래스를 over sampling 해서 클래스 균형을 맞춤
sample_smote = SMOTE(sampling_strategy=0.2,random_state=42)

In [8]:
# 소수클래스가 230 개라면 다수클래스를 5배 (약 1200개) 로 줄이겠다는 부분
sample_rdundersamp = RandomUnderSampler(sampling_strategy=0.2, random_state=42)

In [9]:
#tomek + CNN 해서 적절히 undersampling 
sample_onesidesel = OneSidedSelection(random_state=42)

In [10]:
# 클래스간 경계가 애매한 샘플을 제거해서 안정적으로 under sampling 
# 7623개 -> 7535개 
sample_tomek = TomekLinks()

In [11]:
# 다수클래스에서 가장 중요한 샘플빼고 다 제거
# 클래스 0 (다수클래스)가 520개 가량으로 줄어서 성능이 낮음 
sample_cnn = CondensedNearestNeighbour(random_state=42)

In [12]:
samplers = { 
    # sampler Dict 에 넣어서 반복문 돌리기 쉽게 처리
    # 각각 smote randomsampler tomek cnn 1sideselection 순
    "SMOTE": sample_smote, 
    "RandUnderSamp": sample_rdundersamp,
    "Tomek": sample_tomek,
    "CNN": sample_cnn,
    "1SideSel": sample_onesidesel
}

In [13]:
for lbl, sampler in samplers.items():
    # 반복문에 samplers 의 아이템을 넣고 출력 
    print(" ____________ ")
    # 샘플링
    X_train_over, y_train_over = sampler.fit_resample(x_train, y_train) #샘플링 된 값: x_train_over, y_train_over
    # 해당 sampler 별로 resample 하여 저장

    # 클래스 분포 출력
    unique_over, counts_over = np.unique(y_train_over, return_counts=True) 
    
    for label, count in zip(unique_over, counts_over):
        print(f"{lbl} {label}: {count}개") #샘플링 한 이후 개수 출력 

    # 모델 정의
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1],)))  # 명시적 입력층, shape로 입력층의 데이터 수를 명시
    model.add(Dense(1, activation='sigmoid'))    # 그 다음 Dense 층
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(learning_rate=0.0015735),
                  metrics=['accuracy'])

    # EarlyStopping & 학습    
    # val_loss 를 기준으로 10번 이상 결과가 안 나아지면 멈추고
    # 가장 좋았던 model 을 기준으로 저장
    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True) 


    # sampling 한 train 값들을 x, y 로 넣기
    # epochs = 100 이고 중간중간 확인용 ( early stopping 에 사용 ) validation set 은 20%
    # CALLBACK 하고 verbose는 일단 0으로 해서 출력이 accuracy 만 되도록 함
    model.fit(X_train_over, y_train_over,
              epochs=100, validation_split=0.2,
              callbacks=[early_stop], verbose=0)

    # 평가
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1) #모델을 평가함 loss 와 acc 사용 
    print(f"{lbl} Test Accuracy: {test_acc:.4f}") # sampling 방법에 따른 acc 를 소숫점 4자리까지 출력

 ____________ 
SMOTE 0: 7623개
SMOTE 1: 1524개
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9174 - loss: 0.2035  
SMOTE Test Accuracy: 0.9149
 ____________ 
RandUnderSamp 0: 1145개
RandUnderSamp 1: 229개
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5743 - loss: 4.8784 
RandUnderSamp Test Accuracy: 0.5665
 ____________ 
Tomek 0: 7535개
Tomek 1: 229개
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9796 - loss: 0.0824  
Tomek Test Accuracy: 0.9735
 ____________ 
CNN 0: 519개
CNN 1: 229개
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8763 - loss: 0.3354
CNN Test Accuracy: 0.8701
 ____________ 
1SideSel 0: 7374개
1SideSel 1: 229개
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9772 - loss: 12.1055   
1SideSel Test Accuracy: 0.9710
